In [2]:
import os
import sys
import pandas as pd
import logging


sys.path.append(os.path.abspath('../'))
from src.params import Params
from src.client import DatabaseClient
from src.logging_config import setup_logging

In [3]:
spotify = pd.read_csv("/home/bb-8/Dags/ws_002/data/intermediate/grammys.csv")
grammy = pd.read_csv("/home/bb-8/Dags/ws_002/data/intermediate/spotify.csv")
#api = pd.read_csv("/home/bb-8/Dags/ws_002/data/intermediate/api.csv")